# Desktop DNS Notebook 5

## 4.1 Characteristic Interface tests

Build a regular grid

Match the Visbal & Gaitonde test-case



In [ ]:
# reload modules
%reload_ext autoreload
%autoreload 2

In [660]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from meshing.write_case import *

# pop-out figures
%matplotlib qt
#%matplotlib inline

cfl = 0.5
Lx = 8.0
Ly = 8.0


# case names and resolution
basedir = 'notebook5/mb_interface_test/'
cases = {'a','b','c','d','e','f','g','h','i','j'}
Ni = [75,100,40,125,150,35,60,175,25,50]
Nj = [75,100,40,125,150,35,60,175,25,50]


# inilitialize dictionaries
mesh   = {} 
bcs    = {}
gas    = {}
solver = {}
case   = {}

# solver inputs [SI-units]

# boundary conditions
bcs['Toin'] = 287.57 #K, initial guess of inlet total temp.
bcs['Poin'] = 100000.0 #Pa, initial guess of inlet total p.
bcs['pexit'] = 100000.0 #Pa, exit static p. 
bcs['vin'] = 1 #m/s, inlet velocity
bcs['alpha'] = 0.0 #degrees, inlet yaw angle  
bcs['gamma'] = 0.0 #degrees, inlet pitch angle
bcs['aturb'] = 0.0 # amplification factor for turbulence
bcs['lturb'] = 0 # not used
bcs['ilength'] = 500 # inlet turbulence generation frequency (no. steps)
bcs['radprof'] = 0 # impose spanwise profile when = 1.
bcs['twall'] = -1.0 # not used
bcs['cax'] = 1.0 # not used

# gas properties
gas['gamma']=1.4   # adiabatic exponent
gas['cp'] = 1005.0 # J/kg/K
gas['mu_ref'] = 0.0 # scale ref. viscosity to match Re
gas['mu_tref'] = 273.0 # Sutherland's Tref
gas['mu_cref'] = 110.4 # Sutherland's Cref
gas['pr'] = 0.72 # Prandtl number

# solver inputs
solver['niter'] = np.floor(16.0/dt) # for t=16s
solver['nwrite'] = 1000
solver['ncut'] = 50   
solver['cfl'] = cfl
solver['sigma'] = 0.005 # filter coefficient
    
solver['irestart'] = 1 # re-start
solver['span'] = 0.2 # spanwise extent for 3D calcs - typically 10-20% of chord
solver['fexpan'] = 1.0 # must be 1.0

solver['nk'] = 1 # spanwise points for 3D calculation 
solver['npp'] = 1 # not used for gpu version
solver['istats'] = 0 # 0=no time averaging, 1=3D time average, 2=2D time average
solver['version'] = 'gpu' # write inputs for gpu version of the solver
                
# define block groups for each GPU card
block_groups = 1*[None] # 1 block group 
# blocks in each block group
block_groups[0] = {}
block_groups[0]['blocks'] = [1]
solver['block_groups'] = block_groups
solver['nkproc'] = 1 # GPUs in spanwise direction
        
# Define connectivity    
next_block = 1*[None]
next_patch = 1*[None]

# Block 1
next_block[0] = {}
next_patch[0] = {}
next_block[0]['im'] = 1 # periodic to self
next_block[0]['ip'] = 1
next_block[0]['jm'] = 1 # periodic to self
next_block[0]['jp'] = 1
next_patch[0]['im'] = 2
next_patch[0]['ip'] = 1
next_patch[0]['jm'] = 4
next_patch[0]['jp'] = 3


for i,case in enumerate(cases):
    casename = basedir+case
    ni = Ni[i]
    nj = Nj[j]

    # Find time step to determine no. steps
    dt = cfl*(16.0/np.float(ni-1))/341

    # Create a regular grid
    blk = 1*[None]

    x = np.linspace(-Lx,Lx,ni)
    y = np.linspace(-Ly,Ly,nj)

    xb, yb = np.meshgrid(x, y,  indexing='ij')

    blk[0] = {}
    blk[0]['x'] = xb
    blk[0]['y'] = yb
    

    corner = 1*[None]
    corner[0] = {}
    corner[0]['block'] = [1,1,1,1]
    corner[0]['i'] = [1 ,1 ,ni,ni]
    corner[0]['j'] = [1 ,nj,1 ,nj]
    corner[0]['Nb']=4

    
    case['casename'] = casename
    case['next_block'] = next_block
    case['next_patch'] = next_patch
    case['corner'] = corner
    case['blk'] = blk
    case['solver'] = solver
    case['bcs'] = bcs
    case['gas'] = gas


    # write new case files
    write_case(case)

0.0003976340772404195
Writing inputs for 3dns_gpu
Total ij points
3600
Total GPUs
1


In [9]:
from meshing.write_flo import *


casename = 'notebook5/mb_interface_test/a'

# create an initial flow
cinf = 340.0
Minf = 0.1
Uinf = Minf*cinf
ro = 1.225
R = 1.0
pinf = 1.0e5
Toinf = pinf/(287.14*ro)

C = 0.02*Uinf*R
xc = 0.0
yc = 0.0
R2 = R*R

prop = len(blk)*[None]
for i in range(len(blk)):
    xb = blk[i]['x']
    yb = blk[i]['y']
    r2 = ((xb-xc)*(xb-xc) + (yb-yc)*(yb-yc))/R2
    #u = Uinf - C*(yb-yc)*(np.exp(-r2*0.5))/R2
    u = 1.0 - C*(yb-yc)*(np.exp(-r2*0.5))/R2
    v = C*(xb-xc)*(np.exp(-r2*0.5))/R2
    p = pinf - ro*C*C*0.5*(np.exp(-r2))/R2
    
    prop[i] = {}
    prop[i]['ro'] = ro*np.ones([ni,nj])
    prop[i]['ru'] = ro*u
    prop[i]['rv'] = ro*v
    prop[i]['rw'] = np.zeros([ni,nj])
    prop[i]['Et'] = p/0.4 + 0.5*(u*u + v*v)*ro



# write initial flow files
write_flo(prop,casename)

# set restart flag and re-write case files
case['solver']['irestart'] = 1 # 

# write new case files
write_case(case)

NameError: name 'blk' is not defined

In [18]:
import matplotlib.pyplot as plt
from postpro.read_flo import *


casename = 'notebook5/mb_interface_test/i'

prop,geom=read_flo(casename)

# plot flow
plt.figure(1)
plt.axis('equal')
    
for ib in range(len(geom)):
    x=geom[ib]['x']
    y=geom[ib]['y']       
    #x=blk[ib]['x']
    #y=blk[ib]['y']       
    
    #plt.pcolormesh(x,y, prop[ib]['ro'][:,:,0],shading='gouraud')
    #plt.clim([1.224,1.226])
    plt.pcolormesh(x,y, prop[ib]['rv'][:,:,0],shading='gouraud')
    
    #plt.pcolormesh(x,y, prop[ib]['ru'][:,:,0],shading='gouraud')
    plt.clim([-0.1,0.1])
    #plt.clim([ro*Uinf-0.1,ro*Uinf+0.1])
    #plt.clim([ro-0.1,ro+0.1])
    plt.set_cmap('seismic')
    plt.colorbar()
plt.show()
    


gpu input file found
1


In [60]:

# pop-out figures
%matplotlib qt
#%matplotlib inline

import matplotlib.pyplot as plt
from postpro.read_flo import *

prop = {}
geom = {}
cases = {}


basedir = 'notebook5/mb_interface_test/'

cases = {'a','b','c','d','e','f','g','h','i','j'}
err_mean = np.zeros([len(cases)])
points = np.zeros([len(cases)])
err_case = np.zeros([len(cases)]) 
dx = np.zeros([len(cases)])

# plot flow
yplot = 0.0

# initial flow constants
cinf = 340.0
Minf = 0.1
Uinf = Minf*cinf
ro = 1.225
R = 1.0
pinf = 1.0e5
Toinf = pinf/(287.14*ro)
C = 0.02*Uinf*R
xc = 0.0
yc = 0.0
R2 = R*R

plt.figure(1)
for i,case in enumerate(cases):
    prop,geom=read_flo(basedir+case)
    print(case)
    err_max = -1.0e12
    for ib in range(len(geom)):
        xb=geom[ib]['x']
        yb=geom[ib]['y']
        ni,nj = np.shape(xb)
        r2 = ((xb-xc)*(xb-xc) + (yb-yc)*(yb-yc))/R2
        # compute initial flow for comparison
        u = Uinf - C*(yb-yc)*(np.exp(-r2*0.5))/R2
        v = C*(xb-xc)*(np.exp(-r2*0.5))/R2
        p = pinf - ro*C*C*0.5*(np.exp(-r2))/R2
        
        jplot = np.argmin( (yb[0,:] - yplot)*(yb[0,:] - yplot) )   
        xplot = xb[:,jplot]
        
        vpred =  prop[ib]['rv'][:,:,0]/prop[ib]['ro'][:,:,0]
        err = np.max(np.abs(v[:,jplot]-vpred[:,jplot]))
        
        if(err > err_max):
            err_max = np.max(err)
        
        points[i] = points[i] + ni*nj
        plt.plot(xplot, 100*(vpred[:,jplot]-v[:,jplot])/(C*R*np.exp(-0.5)),'-k.') 
        plt.ylabel('swirl velocity (% of max)')
        #plt.pcolormesh(xb,yb,err)
        #plt.clim([0,0.01])
    err_case[i] = err_max
    dx[i] = 16.0/np.float(ni-1)
    print('error in v = ',err_max)
    
plt.show()


#err_mean = (err_mean/points)

points = np.sqrt(points)

inorm = np.argmin(np.abs(points-25))

pts = np.linspace(1,9,10)


model = np.poly1d(np.polyfit(np.log(points/points[inorm]),np.log(err_case/err_case[inorm]), 1))
yfit = np.exp(model(np.log(pts)))

#plt.figure(2)
#plt.plot(dx,np.log(err_case),'ks')
#plt.plot(dx,np.log(err_case[0]*((dx/dx[0])**4.0)),'bx')
#plt.plot(dx,np.log(err_case[0]*((dx/dx[0])**3.0)),'gx')
#plt.plot(dx,np.log(err_case[0]*((dx/dx[0])**2.0)),'rx')

#plt.axis([0.1,1.0,-5.0,-1.5])


plt.figure(10)
plt.loglog(points/points[inorm],err_case/err_case[inorm],'ko')
plt.loglog(pts,(pts)**(-2.0),'--r')
#plt.loglog(pts,(pts)**(-3.0),'--g')
plt.loglog(pts,(pts)**(-4.0),'--b')
plt.loglog(pts,yfit,'--m')


# double precision limit
#plt.loglog(pts,((2.0**(-64))*pts/pts)/err_case[inorm],'--k')

print(model)

plt.show()


gpu input file found
1
a
error in v =  0.005616252170301761
gpu input file found
1
h
error in v =  0.0004949491071925549
gpu input file found
1
i
error in v =  0.20679014006980512
gpu input file found
1
f
error in v =  0.0857373894819293
gpu input file found
1
g
error in v =  0.013082522877672698
gpu input file found
1
b
error in v =  0.002460583233956315
gpu input file found
1
e
error in v =  0.000740016889135775
gpu input file found
1
c
error in v =  0.05875038504735847
gpu input file found
1
j
error in v =  0.025654398575710002
gpu input file found
1
d
error in v =  0.0012454453869512605
 
-3.212 x + 0.09804


## Non-reflecting boundary condition test

### Gaussian pulse

In [52]:
import numpy as np
from meshing.write_case import *
from meshing.write_flo import *
from meshing.write_probe import *

# pop-out figures
%matplotlib qt
#%matplotlib inline

cfl = 0.5

# case name and resolution
casename = 'notebook5/gauss_pulse/default'
ni = 125
nj = 125

# Create a regular grid
blk = 1*[None]
Lx = 50.0
Ly = 50.0

# Find time step to determine no. steps
dt = cfl*(Lx/np.float(ni-1))/341

x = np.linspace(-Lx,Lx,ni)
y = np.linspace(-Ly,Ly,nj)

xb, yb = np.meshgrid(x, y,  indexing='ij')

blk[0] = {}
blk[0]['x'] = xb
blk[0]['y'] = yb

# inilitialize dictionaries
mesh   = {}
bcs    = {}
gas    = {}
solver = {}
case   = {}


# solver inputs [SI-units]

# boundary conditions
bcs['Toin']  = 287.57 #K, initial guess of inlet total temp.
bcs['Poin']  = 100000.0 + 1.21*0.5 #Pa, initial guess of inlet total p.
bcs['pexit'] = 100000.0 #Pa, exit static p. 
bcs['vin']   = 1.0 #m/s, inlet velocity
bcs['alpha'] = 0.0 #degrees, inlet yaw angle  
bcs['gamma'] = 0.0 #degrees, inlet pitch angle
bcs['aturb'] = 0.0 # amplification factor for turbulence
bcs['lturb'] = 0 # not used
bcs['ilength'] = 500 # inlet turbulence generation frequency (no. steps)
bcs['radprof'] = 0 # impose spanwise profile when = 1.
bcs['twall'] = -1.0 # not used
bcs['cax'] = 1.0 # not used


# gas properties
gas['gamma']=1.4   # adiabatic exponent
gas['cp'] = 1005.0 # J/kg/K
gas['mu_ref'] = 0.0 # scale ref. viscosity to match Re
gas['mu_tref'] = 273.0 # Sutherland's Tref
gas['mu_cref'] = 110.4 # Sutherland's Cref
gas['pr'] = 0.72 # Prandtl number


# solver inputs
solver['niter'] = 10*(2.0*Lx/341)/dt 
solver['nwrite'] = 1000
solver['ncut'] = 10   
solver['cfl'] = cfl
solver['sigma'] = 0.03 # filter coefficient
    
solver['irestart'] = 1 # re-start
solver['span'] = 0.2 # spanwise extent for 3D calcs - typically 10-20% of chord
solver['fexpan'] = 1.0 # must be 1.0

solver['nk'] = 1 # spanwise points for 3D calculation 
solver['npp'] = 1 # not used for gpu version
solver['istats'] = 0 # 0=no time averaging, 1=3D time average, 2=2D time average
solver['version'] = 'gpu' # write inputs for gpu version of the solver
                
# define block groups for each GPU card
block_groups = 1*[None] # 1 block group 
# blocks in each block group
block_groups[0] = {}
block_groups[0]['blocks'] = [1]
solver['block_groups'] = block_groups
solver['nkproc'] = 1 # GPUs in spanwise direction

total_GPUs = solver['nkproc']*len(block_groups) # total number of GPUs
         
# Define connectivity    
next_block = 1*[None]
next_patch = 1*[None]

# Block 1
next_block[0] = {}
next_patch[0] = {}
next_block[0]['im'] = 0 # inlet
next_block[0]['ip'] = 0 # exit
next_block[0]['jm'] = 1 # periodic to self
next_block[0]['jp'] = 1
next_patch[0]['im'] = 1
next_patch[0]['ip'] = 2
next_patch[0]['jm'] = 4
next_patch[0]['jp'] = 3
    

corner = []

case['casename'] = casename
case['next_block'] = next_block
case['next_patch'] = next_patch
case['corner'] = corner
case['blk'] = blk
case['solver'] = solver
case['bcs'] = bcs
case['gas'] = gas


probe = {}
probe['nprobe'] = 1 # 1 probes
probe['nwrite'] = 1 # store every iterations

# probe 1 - near exit centre-line
probe[0] = {}
probe[0]['block'] = 1 # block number 
probe[0]['i'] = 110 # i value
probe[0]['j'] = 63 # j value
probe[0]['k'] = 1  # k value

# create an initial flow
cinf = 340.0
Minf = 0.1
Uinf = Minf*cinf
R = 1.0
pinf = 1.0e5
Tinf = 287.57
roinf = pinf/(287.14*Tinf)
eps = 1.0e-4

prop = len(blk)*[None]
for i in range(len(blk)):
    xb = blk[i]['x']
    yb = blk[i]['y']
    u = np.ones([ni,nj]) #np.zeros([ni,nj])
    v = np.zeros([ni,nj])
    w = np.zeros([ni,nj])
    p = pinf - eps*roinf*cinf*cinf*np.exp( -np.log(2.0)*(xb*xb)/(9.0*R*R) ) 
    u = u + (p-pinf)/(roinf*cinf)
    ro = ((p/pinf)**(1.0/1.4))*roinf
    
    prop[i] = {}
    prop[i]['ro'] = ro*np.ones([ni,nj])
    prop[i]['ru'] = ro*u
    prop[i]['rv'] = ro*v
    prop[i]['rw'] = ro*w
    prop[i]['Et'] = p/0.4 + 0.5*(u*u + v*v)*ro
    prop[i]['p'] = p
    
# set restart flag and re-write case files
case['solver']['irestart'] = 1 # 

# write new case files
write_case(case)

# write initial flow files
write_flo(prop,casename)

# write probe.txt file
write_probe(casename,probe)



Writing inputs for 3dns_gpu
Total ij points
15625
Total GPUs
1
gpu input file found
1


In [27]:
dp = eps*roinf*cinf*cinf
for ib in range(len(blk)):
    x=blk[ib]['x']
    y=blk[ib]['y']       
    
    plt.pcolormesh(x,y, prop[ib]['p'][:,:],shading='gouraud') 
    plt.clim([pinf-dp,pinf+dp])
    plt.set_cmap('seismic')
    plt.colorbar()
plt.show()

### Read probe data

In [59]:
import matplotlib.pyplot as plt
import numpy as np
from postpro.read_probe import *


casename = 'notebook5/gauss_pulse/default'

probe = read_probe(casename)

plt.figure(1)
# plot the probe data
print('Probe position')
print('x = ', probe[0]['x'])
print('y = ', probe[0]['y'])
plt.plot(probe[0]['time'],probe[0]['p'],'-k')
plt.xlabel('Time (s)')
plt.ylabel('Pressure (Pa)')
    
plt.show()

time = probe[0]['time']
p = probe[0]['p']
ro = probe[0]['ro']
u = probe[0]['u']
c = probe[0]['c']

pp = p - np.mean(p)
up = u - np.mean(u)
rp =ro - np.mean(ro)

# left and right chics
wm = pp - ro*c*up
wp = pp + ro*c*up

plt.figure(2)

plt.plot(time,wm,'-b',time,wp,'-r',)
plt.xlabel('Time (s)')
plt.ylabel('Chic')
    
plt.show()





gpu input file found
1
Probe position
x =  37.9032258064516
y =  0.0


In [55]:
from postpro.read_kcuts import *

dp = eps*roinf*cinf*cinf

read_kcuts('notebook5/gauss_pulse/default',[1000],'p',[pinf-dp,pinf+dp]);


gpu input file found
1


### Adjust the inlet and exit stiffness

In [886]:
from meshing.write_buffer import *

inlet  = {}
outlet = {}

inlet['N'] = 10
inlet['Lwave'] = 1.0


outlet['N'] = 10
outlet['Lwave'] = 1.0

write_buffer(casename,inlet,outlet)


### Explore oblique waves

In [47]:
import numpy as np
from meshing.write_case import *
from meshing.write_flo import *
from meshing.write_probe import *

# pop-out figures
%matplotlib qt
#%matplotlib inline

cfl = 0.5

# case name and resolution
casename = 'notebook5/gauss_pulse/curvi'
ni = 125
nj = 125

# Create a regular grid
blk = 1*[None]
Lx = 50.0
Ly = 50.0

# Find time step to determine no. steps
dt = cfl*(Lx/np.float(ni-1))/341

# Create curved mesh
amp = Lx*0.5
x = np.linspace(-Lx,Lx,ni)
y = np.linspace(-Ly,Ly,nj)


xb, yb = np.meshgrid(x, y,  indexing='ij')
xx = xb
yy = yb

xb = xb + amp*((yy/Ly)**2.0)*(xb-xb[0,0])/(2.0*Lx)
yb = yb

blk[0] = {}
blk[0]['x'] = xb
blk[0]['y'] = yb

# inilitialize dictionaries
mesh   = {}
bcs    = {}
gas    = {}
solver = {}
case   = {}


# solver inputs [SI-units]

# boundary conditions
bcs['Toin']  = 287.57 #K, initial guess of inlet total temp.
bcs['Poin']  = 100000.0 + 1.21*0.5 #Pa, initial guess of inlet total p.
bcs['pexit'] = 100000.0 #Pa, exit static p. 
bcs['vin']   = 1.0 #m/s, inlet velocity
bcs['alpha'] = 0.0 #degrees, inlet yaw angle  
bcs['gamma'] = 0.0 #degrees, inlet pitch angle
bcs['aturb'] = 0.0 # amplification factor for turbulence
bcs['lturb'] = 0 # not used
bcs['ilength'] = 500 # inlet turbulence generation frequency (no. steps)
bcs['radprof'] = 0 # impose spanwise profile when = 1.
bcs['twall'] = -1.0 # not used
bcs['cax'] = 1.0 # not used


# gas properties
gas['gamma']=1.4   # adiabatic exponent
gas['cp'] = 1005.0 # J/kg/K
gas['mu_ref'] = 0.0 # scale ref. viscosity to match Re
gas['mu_tref'] = 273.0 # Sutherland's Tref
gas['mu_cref'] = 110.4 # Sutherland's Cref
gas['pr'] = 0.72 # Prandtl number


# solver inputs
solver['niter'] = 10*(2.0*Lx/341)/dt 
solver['nwrite'] = 1000
solver['ncut'] = 10   
solver['cfl'] = cfl
solver['sigma'] = 0.03 # filter coefficient
    
solver['irestart'] = 1 # re-start
solver['span'] = 0.2 # spanwise extent for 3D calcs - typically 10-20% of chord
solver['fexpan'] = 1.0 # must be 1.0

solver['nk'] = 1 # spanwise points for 3D calculation 
solver['npp'] = 1 # not used for gpu version
solver['istats'] = 0 # 0=no time averaging, 1=3D time average, 2=2D time average
solver['version'] = 'gpu' # write inputs for gpu version of the solver
                
# define block groups for each GPU card
block_groups = 1*[None] # 1 block group 
# blocks in each block group
block_groups[0] = {}
block_groups[0]['blocks'] = [1]
solver['block_groups'] = block_groups
solver['nkproc'] = 1 # GPUs in spanwise direction

total_GPUs = solver['nkproc']*len(block_groups) # total number of GPUs
         
# Define connectivity    
next_block = 1*[None]
next_patch = 1*[None]

# Block 1
next_block[0] = {}
next_patch[0] = {}
next_block[0]['im'] = 0 # inlet
next_block[0]['ip'] = 0 # exit
next_block[0]['jm'] = 1 # periodic to self
next_block[0]['jp'] = 1
next_patch[0]['im'] = 1
next_patch[0]['ip'] = 2
next_patch[0]['jm'] = 4
next_patch[0]['jp'] = 3
    

corner = []

case['casename'] = casename
case['next_block'] = next_block
case['next_patch'] = next_patch
case['corner'] = corner
case['blk'] = blk
case['solver'] = solver
case['bcs'] = bcs
case['gas'] = gas


probe = {}
probe['nprobe'] = 1 # 1 probes
probe['nwrite'] = 1 # store every iterations

# probe 1 - near exit centre-line
probe[0] = {}
probe[0]['block'] = 1 # block number 
probe[0]['i'] = 100 # i value
probe[0]['j'] = 63 # j value
probe[0]['k'] = 1  # k value


# create an initial flow
cinf = 340.0
Minf = 0.1
Uinf = Minf*cinf
R = 1.0
pinf = 1.0e5
Tinf = 287.57
roinf = pinf/(287.14*Tinf)
eps = 1.0e-4
#eps = 1.0e-4


plt.figure(2)
plt.axis('equal')
prop = len(blk)*[None]
for i in range(len(blk)):
    xb = blk[i]['x']
    yb = blk[i]['y']
    u = np.ones([ni,nj]) #np.zeros([ni,nj])
    v = np.zeros([ni,nj])
    w = np.zeros([ni,nj])
    p = pinf - eps*roinf*cinf*cinf*np.exp( -np.log(2.0)*(xb*xb)/(9.0*R*R) ) 
    u = u + (p-pinf)/(roinf*cinf)
    #p = pinf - eps*roinf*cinf*cinf*np.exp( -np.log(2.0)*(xb*xb + yb*yb)/(9.0*R*R) ) 
    ro = ((p/pinf)**(1.0/1.4))*roinf
  
    prop[i] = {}
    prop[i]['ro'] = ro*np.ones([ni,nj])
    prop[i]['ru'] = ro*u
    prop[i]['rv'] = ro*v
    prop[i]['rw'] = ro*w
    prop[i]['Et'] = p/0.4 + 0.5*(u*u + v*v)*ro
    plt.pcolormesh(xb,yb, prop[ib]['ro'],shading='gouraud')
    plt.plot(xb,yb,xb.T,yb.T)
   
    
# set restart flag and re-write case files
case['solver']['irestart'] = 1 # 

# write new case files
write_case(case)

# write initial flow files
write_flo(prop,casename)

# write probe.txt file
write_probe(casename,probe)


Writing inputs for 3dns_gpu
Total ij points
15625
Total GPUs
1
gpu input file found
1


In [49]:
import matplotlib.pyplot as plt
import numpy as np
from postpro.read_probe import *

probe = read_probe(casename)

plt.figure(1)
# plot the probe data
print('Probe position')
print('x = ', probe[0]['x'])
print('y = ', probe[0]['y'])
plt.plot(probe[0]['time'],probe[0]['p'],'-k')
plt.xlabel('Time (s)')
plt.ylabel('Pressure (Pa)')
    
plt.show()

time = probe[0]['time']
p = probe[0]['p']
ro = probe[0]['ro']
u = probe[0]['u']
v = probe[0]['v']
c = probe[0]['c']

pp = p - np.mean(p)
up = u - np.mean(u)
vp = v - np.mean(v)
rp =ro - np.mean(ro)

# left and right chics
wm = pp - ro*c*up
wp = pp + ro*c*up

## up and down chics
#wu = pp - ro*c*vp
#wd = pp + ro*c*vp

plt.figure(2)

plt.plot(time,wm,'-b',time,wp,'-r')
plt.xlabel('Time (s)')
plt.ylabel('Chic')
    
plt.show()

gpu input file found
1
Probe position
x =  29.838709677419345
y =  0.0


gpu input file found
1
Writing inputs for 3dns_gpu
Total ij points
5625
Total GPUs
1


In [48]:
import matplotlib.pyplot as plt
from postpro.read_flo import *

prop,geom=read_flo(casename)

# plot flow
plt.figure(1)
plt.axis('equal')
    
for ib in range(len(geom)):
    x=geom[ib]['x']
    y=geom[ib]['y']       
    #x=blk[ib]['x']
    #y=blk[ib]['y']       
    
    #plt.pcolormesh(x,y, prop[ib]['ro'][:,:,0],shading='gouraud')
    #plt.clim([1.224,1.226])
    plt.pcolormesh(x,y, prop[ib]['u'][:,:,0],shading='gouraud')
    
    #plt.pcolormesh(x,y, prop[ib]['ru'][:,:,0],shading='gouraud')
    #plt.clim([-0.1,0.1])
    #plt.clim([ro*Uinf-0.1,ro*Uinf+0.1])
    #plt.clim([ro-0.01,ro+0.01])
    plt.colorbar()
plt.show()

gpu input file found
1


In [50]:
from postpro.read_kcuts import *


dp = eps*roinf*cinf*cinf

read_kcuts('notebook5/gauss_pulse/curvi',[1000],'p',[pinf-dp,pinf+dp]);


gpu input file found
1


### Vortex hitting exit

In [56]:

import numpy as np
from meshing.write_case import *

# pop-out figures
%matplotlib qt
#%matplotlib inline

cfl = 0.5

# case name and resolution
casename = 'notebook5/vortex/'
ni = 75
nj = 75

# Find time step to determine no. steps
dt = cfl*(16.0/np.float(ni-1))/341

# Create a regular grid
blk = 1*[None]
Lx = 8.0
Ly = 8.0

x = np.linspace(-Lx,Lx,ni)
y = np.linspace(-Ly,Ly,nj)

xb, yb = np.meshgrid(x, y,  indexing='ij')

blk[0] = {}
blk[0]['x'] = xb
blk[0]['y'] = yb

# inilitialize dictionaries
mesh   = {}
bcs    = {}
gas    = {}
solver = {}
case   = {}


# solver inputs [SI-units]

# boundary conditions
bcs['Toin']  = 287.57 #K, initial guess of inlet total temp.
bcs['Poin']  = 100000.0 + 1.21*0.5 #Pa, initial guess of inlet total p.
bcs['pexit'] = 100000.0 #Pa, exit static p. 
bcs['vin']   = 1.0 #m/s, inlet velocity
bcs['alpha'] = 0.0 #degrees, inlet yaw angle  
bcs['gamma'] = 0.0 #degrees, inlet pitch angle
bcs['aturb'] = 0.0 # amplification factor for turbulence
bcs['lturb'] = 0 # not used
bcs['ilength'] = 500 # inlet turbulence generation frequency (no. steps)
bcs['radprof'] = 0 # impose spanwise profile when = 1.
bcs['twall'] = -1.0 # not used
bcs['cax'] = 1.0 # not used


# gas properties
gas['gamma']=1.4   # adiabatic exponent
gas['cp'] = 1005.0 # J/kg/K
gas['mu_ref'] = 0.0 # scale ref. viscosity to match Re
gas['mu_tref'] = 273.0 # Sutherland's Tref
gas['mu_cref'] = 110.4 # Sutherland's Cref
gas['pr'] = 0.72 # Prandtl number


# solver inputs
solver['niter'] = np.floor(16.0/dt) # for t=16s
solver['nwrite'] = 1000
solver['ncut'] = 100   
solver['cfl'] = cfl
solver['sigma'] = 0.03 # filter coefficient
    
solver['irestart'] = 1 # re-start
solver['span'] = 0.2 # spanwise extent for 3D calcs - typically 10-20% of chord
solver['fexpan'] = 1.0 # must be 1.0

solver['nk'] = 1 # spanwise points for 3D calculation 
solver['npp'] = 1 # not used for gpu version
solver['istats'] = 0 # 0=no time averaging, 1=3D time average, 2=2D time average
solver['version'] = 'gpu' # write inputs for gpu version of the solver
                
# define block groups for each GPU card
block_groups = 1*[None] # 1 block group 
# blocks in each block group
block_groups[0] = {}
block_groups[0]['blocks'] = [1]
solver['block_groups'] = block_groups
solver['nkproc'] = 1 # GPUs in spanwise direction

total_GPUs = solver['nkproc']*len(block_groups) # total number of GPUs
         
# Define connectivity    
next_block = 1*[None]
next_patch = 1*[None]

# Block 1
next_block[0] = {}
next_patch[0] = {}
next_block[0]['im'] = 0 # inlet
next_block[0]['ip'] = 0 # exit
next_block[0]['jm'] = 1 # periodic to self
next_block[0]['jp'] = 1
next_patch[0]['im'] = 1
next_patch[0]['ip'] = 2
next_patch[0]['jm'] = 4
next_patch[0]['jp'] = 3
    

corner = []

case['casename'] = casename
case['next_block'] = next_block
case['next_patch'] = next_patch
case['corner'] = corner
case['blk'] = blk
case['solver'] = solver
case['bcs'] = bcs
case['gas'] = gas


# create an initial flow
cinf = 340.0
Minf = 0.1
Uinf = Minf*cinf
ro = 1.225
R = 1.0
pinf = 1.0e5
Toinf = pinf/(287.14*ro)

C = 0.02*Uinf*R
xc = 0.0
yc = 0.0
R2 = R*R

prop = len(blk)*[None]
for i in range(len(blk)):
    xb = blk[i]['x']
    yb = blk[i]['y']
    r2 = ((xb-xc)*(xb-xc) + (yb-yc)*(yb-yc))/R2
    u = 1.0 - C*(yb-yc)*(np.exp(-r2*0.5))/R2
    v = C*(xb-xc)*(np.exp(-r2*0.5))/R2
    p = pinf - ro*C*C*0.5*(np.exp(-r2))/R2
    
    prop[i] = {}
    prop[i]['ro'] = ro*np.ones([ni,nj])
    prop[i]['ru'] = ro*u
    prop[i]['rv'] = ro*v
    prop[i]['rw'] = np.zeros([ni,nj])
    prop[i]['Et'] = p/0.4 + 0.5*(u*u + v*v)*ro


# set restart flag and re-write case files
case['solver']['irestart'] = 1 # 

# write new case files
write_case(case)

# write initial flow files
write_flo(prop,casename)


Writing inputs for 3dns_gpu
Total ij points
5625
Total GPUs
1
gpu input file found
1


In [57]:
import matplotlib.pyplot as plt
from postpro.read_flo import *

prop,geom=read_flo(casename)

# plot flow
plt.figure(1)
plt.axis('equal')
    
for ib in range(len(geom)):
    x=geom[ib]['x']
    y=geom[ib]['y']       
    #x=blk[ib]['x']
    #y=blk[ib]['y']       
    
    #plt.pcolormesh(x,y, prop[ib]['ro'][:,:,0],shading='gouraud')
    #plt.clim([1.224,1.226])
    plt.pcolormesh(x,y, prop[ib]['vortz'][:,:,0],shading='gouraud')
    
    #plt.pcolormesh(x,y, prop[ib]['ru'][:,:,0],shading='gouraud')
    #plt.clim([-0.1,0.1])
    #plt.clim([ro*Uinf-0.1,ro*Uinf+0.1])
    #plt.clim([ro-0.01,ro+0.01])
    plt.colorbar
plt.show()

gpu input file found
1


In [58]:
from postpro.read_kcuts import *


read_kcuts('notebook5/vortex',[1000],'vortz',[-0.1,0.1]);


gpu input file found
1


### Create video

Run ffmpeg to create a video

> ffmpeg -i kcut_%d.png vortex.mp4

Play the video
> ffplay vortex.mp4 
